In [3]:
# from transformers import AutoTokenizer

# checkpoint = "google-t5/t5-small"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)



import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device="cuda" if torch.cuda.is_available() else "cpu"

model_name="distilbert/distilgpt2"
# model_name= "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoded_input = tokenizer("Hugging Face is an open-source company", return_tensors="pt").to(device)


model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)


eos = tokenizer(tokenizer.eos_token, return_tensors="pt", return_token_type_ids=False).input_ids.flatten()[0].item()
bos = tokenizer(tokenizer.bos_token, return_tensors="pt", return_token_type_ids=False).input_ids.flatten()[0].item()


x=encoded_input.input_ids
wrapped_tokenizer=tokenizer



In [7]:




# print(f"start_gamma is {start_gamma}")
# print(f"best_y is {best_y}")
# print(f"Gamma is ",gamma)






start_gamma is -9.8046875
best_y is tensor([[50256]])
Gamma is  0
gen_outputs  0  :  


0

In [13]:



with torch.no_grad():
    output = model(full_sentence)
    sequences = encoded_input["input_ids"]
    logits = output.logits


for completion_id in range(num_generated):
    generated_tokens_logprobs = []
    for i in range(len(sequences[completion_id]) - len(encoded_input.input_ids[0])):
        logprobs = torch.nn.functional.log_softmax(logits[i][completion_id], dim=0)
        # Get log probability of chosen token.
        j = i + len(encoded_input.input_ids[0])
        generated_tokens_logprobs.append(logprobs[sequences[completion_id][j]].item())
    all_generated_tokens_logprobs.append(generated_tokens_logprobs)

return sequences, logits

torch.Size([1, 9])
tensor([[50256]])
tensor([[[-31.8906, -30.0625, -31.7188,  ..., -43.2500, -42.9688, -31.8906],
         [-62.5938, -63.3750, -66.9375,  ..., -71.0000, -70.6250, -65.1875],
         [-61.0938, -61.1562, -64.1875,  ..., -69.8125, -67.8750, -61.7500],
         ...,
         [-63.8750, -63.3438, -66.3125,  ..., -70.1875, -69.0000, -64.7500],
         [-62.9062, -65.1250, -69.5000,  ..., -74.6250, -73.3125, -64.1250],
         [-51.7500, -44.7500, -47.1875,  ..., -65.4375, -65.5000, -53.1250]]],
       dtype=torch.float16)
